<a href="https://colab.research.google.com/github/melikesentuk/steps/blob/main/RFM_ile_musteri_segmentasyonu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.iş problemi


In [1]:
#Bir e-ticaret şirketi müşterilerini segmentlere ayırıp bu segmentlere göre
#pazarlama stratejileri belirlemek istiyor.

# kütüphaneleri yükleyelim
import pandas as pd
import datetime as dt

In [2]:
# çıktıda tüm satır ve sütünları görmek için ayarlar
pd.set_option('display.max_columns',None)

pd.set_option('display.max_rows',None)  #tüm satır ayarını aktifleştimredim
# sayısal eğerler için virgülden sonra 3 basamak gösterilsin
pd.set_option('display.float_format',lambda x: '%.3f' % x)

In [4]:
# veri setini yükleyelim
df_ = pd.read_excel("/content/online_retail_2.xlsx", sheet_name='Year 2009-2010')

In [6]:
df =df_.copy()

In [7]:
# veri setiyle ilgili bilgi edinelim
df.head()
#invoice : fatura numarası

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.950,13085.000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.100,13085.000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.250,13085.000,United Kingdom


In [8]:
df.shape

(525461, 8)

In [9]:
df.isnull().sum()
# customer ıdsi eksik olanlara rfm analizi yapamayız 

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

In [10]:
df['Description'].nunique()

4681

In [11]:
df['Description'].value_counts().head()

WHITE HANGING HEART T-LIGHT HOLDER    3549
REGENCY CAKESTAND 3 TIER              2212
STRAWBERRY CERAMIC TRINKET BOX        1843
PACK OF 72 RETRO SPOT CAKE CASES      1466
ASSORTED COLOUR BIRD ORNAMENT         1457
Name: Description, dtype: int64

In [12]:
from numpy import quantile
# eşsiz ürünlerden ne kadar satılmış
df.groupby('Description').agg({"Quantity":"sum"}).head()

# - değerler ar. muhtemelen iade olduğundan ileride bunudüzeltiriz.
# oyüzden satış miktarına göre sıralama  yapalım

,Quantity
Description,
21494,-720
22467,-2
22719,2
DOORMAT UNION JACK GUNS AND ROSES,179
3 STRIPEY MICE FELTCRAFT,690


In [13]:
df.groupby('Description').agg({'Quantity':'sum'}).sort_values('Quantity',ascending=False).head()

,Quantity
Description,
WHITE HANGING HEART T-LIGHT HOLDER,57733
WORLD WAR 2 GLIDERS ASSTD DESIGNS,54698
BROCADE RING PURSE,47647
PACK OF 72 RETRO SPOT CAKE CASES,46106
ASSORTED COLOUR BIRD ORNAMENT,44925


In [14]:
# kaç fatura kesilmiş
df['Invoice'].nunique()

28816

In [15]:
# kesilen bu faturaların toplamı için quantity * price değerlerini bulmak gerek
df["q_p"]=df["Quantity"]*df["Price"]
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,q_p
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.950,13085.000,United Kingdom,83.400
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom,81.000
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom,81.000
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.100,13085.000,United Kingdom,100.800
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.250,13085.000,United Kingdom,30.000


In [16]:
# fatura başına toplam ödenen( müşterinin ödediği) para
df.groupby('Invoice').agg({'q_p':'sum'}).head()

,q_p
Invoice,
489434,505.300
489435,145.800
489436,630.330
489437,310.750
489438,2286.240


VERİ HAZIRLAMA

In [17]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
q_p                 0
dtype: int64

In [18]:
# customer ıd descriptino da boş değerler var, onları silelim
df.dropna(inplace=True)
df.isnull().sum()

# aykırı değer analiziyapmayı tercih etmedim çünkü o değerler rfm analizinde 5 e karşılık gelecek

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
q_p            0
dtype: int64

In [19]:
df.size

3757806

In [20]:
# iade olan değerleri veri setinden silelim
# önce c li değerleri görelim / sayısı
df[df['Invoice'].str.contains("C", na=False)].nunique()


Invoice        4372
StockCode      2080
Description    2194
Quantity        157
InvoiceDate    4292
Price           431
Customer ID    1798
Country          26
q_p            1119
dtype: int64

In [21]:
# İnvoice da c içermeyenleri görelim
df  = df[~df['Invoice'].str.contains('C',na=False)]

In [22]:
df.size   # silmeden önce 3757806 idi

3669255

# **RFM  metriklerinin  Hesaplanması**
R  = analizin yapıldığı  tarih - sonsatın alma yapılan tarih
F = müşterinin yaptığı topkam satın alma
M = yapılan satın almanın parasal değeri

In [23]:
# veri setindeki en son tarih 
df['InvoiceDate'].max()
# +2 günle analiz tarihi verelim
today_date= dt.datetime(2010,12,11)

In [24]:
rfm = df.groupby('Customer ID').agg({'InvoiceDate': lambda date:(today_date - date.max()).days,
                                   'Invoice': lambda num: num.nunique(),
                                   'q_p': lambda q_p: q_p.sum()})

In [25]:
rfm.head()


,InvoiceDate,Invoice,q_p
Customer ID,,,
12346.000,165,11,372.860
12347.000,3,2,1323.320
12348.000,74,1,222.160
12349.000,43,3,2671.140
12351.000,11,1,300.930


In [26]:
rfm.shape

(4314, 3)

In [27]:
rfm.columns=['Recency','Frequency','Monetary']
rfm.head()

,Recency,Frequency,Monetary
Customer ID,,,
12346.000,165,11,372.860
12347.000,3,2,1323.320
12348.000,74,1,222.160
12349.000,43,3,2671.140
12351.000,11,1,300.930


In [28]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
Recency,4314.000,91.270,96.944,1.000,18.000,53.000,136.000,374.000
Frequency,4314.000,4.454,8.169,1.000,1.000,2.000,5.000,205.000
Monetary,4314.000,2047.289,8912.523,0.000,307.950,705.550,1722.802,349164.350


In [29]:
# monetary değeri min 0 olanlar var o değerleri çıkartalım
rfm = rfm[rfm['Monetary']>0]

In [30]:
rfm.size  # 2 değer varmış 0 a eşit yada küçük

12936

In [31]:
rfm.shape

(4312, 3)

In [32]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
Recency,4312.000,91.173,96.861,1.000,18.000,53.000,136.000,374.000
Frequency,4312.000,4.456,8.170,1.000,1.000,2.000,5.000,205.000
Monetary,4312.000,2048.238,8914.481,2.950,307.988,706.020,1723.142,349164.350


RFM değerlerinin hesaplanması

In [33]:
#  r de değerin  küçükolması, f ve m nin büyük olmasını istiyoruz

rfm['Recency_score'] = pd.qcut(rfm['Recency'],5,labels=[5,4,3,2,1])
rfm['Frequency_score'] = pd.qcut(rfm['Frequency'].rank(method="first"), 5, labels=[1,2,3,4,5])
rfm['Monetary_score'] = pd.qcut(rfm['Monetary'], 5, labels=[1,2,3,4,5])

In [34]:
rfm.columns

Index(['Recency', 'Frequency', 'Monetary', 'Recency_score', 'Frequency_score',
       'Monetary_score'],
      dtype='object')

In [35]:
# r ve f değerlerini bir araya getirmemiz gerekiyor
rfm["RFM_SCORE"] = (rfm["Recency_score"].astype(str) +
                    rfm["Frequency_score"].astype(str))

In [36]:
rfm.head()

,Recency,Frequency,Monetary,Recency_score,Frequency_score,Monetary_score,RFM_SCORE
Customer ID,,,,,,,
12346.000,165,11,372.860,2,5,2,25
12347.000,3,2,1323.320,5,2,4,52
12348.000,74,1,222.160,2,1,1,21
12349.000,43,3,2671.140,3,3,5,33
12351.000,11,1,300.930,5,1,2,51


In [37]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
Recency,4312.000,91.173,96.861,1.000,18.000,53.000,136.000,374.000
Frequency,4312.000,4.456,8.170,1.000,1.000,2.000,5.000,205.000
Monetary,4312.000,2048.238,8914.481,2.950,307.988,706.020,1723.142,349164.350


In [38]:
# şampiyon müsteriler ==rf değeri 55 olanlar
rfm[rfm["RFM_SCORE"] == "55"]

,Recency,Frequency,Monetary,Recency_score,Frequency_score,Monetary_score,RFM_SCORE
Customer ID,,,,,,,
12415.000,11,7,19543.840,5,5,5,55
12431.000,9,13,4370.520,5,5,5,55
12471.000,10,49,20139.740,5,5,5,55
12472.000,5,13,11308.480,5,5,5,55
12474.000,14,13,5048.660,5,5,5,55
12483.000,14,11,2635.600,5,5,5,55
12490.000,14,11,4405.710,5,5,5,55
12523.000,10,12,2582.380,5,5,5,55
12540.000,14,16,8854.250,5,5,5,55


RFM SEGMENTLERİNİN OLUŞTURULMASI VE ANALİZ  EDİLMESİ

In [43]:
# RFM isimlendirmesi
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At_Risk',
    r'[1-2]5': 'Cant_loose',
    r'3[1-2]': 'About_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

In [44]:
rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)

In [45]:
rfm.head()

,Recency,Frequency,Monetary,Recency_score,Frequency_score,Monetary_score,RFM_SCORE,segment
Customer ID,,,,,,,,
12346.000,165,11,372.860,2,5,2,25,Cant_loose
12347.000,3,2,1323.320,5,2,4,52,potential_loyalists
12348.000,74,1,222.160,2,1,1,21,Hibernating
12349.000,43,3,2671.140,3,3,5,33,need_attention
12351.000,11,1,300.930,5,1,2,51,new_customers


In [50]:
# elde ettğimiz sınıfların analizini yapalım
rfm[["segment", "Recency", "Frequency", "Monetary"]].groupby("segment").agg(["mean","count"])

Recency       Frequency       Monetary      
                       mean count      mean count     mean count
segment                                                         
About_to_sleep       53.819   343     1.201   343  441.320   343
At_Risk             152.159   611     3.074   611 1188.878   611
Cant_loose          124.117    77     9.117    77 4099.450    77
Hibernating         213.886  1015     1.126  1015  403.978  1015
champions             7.119   663    12.554   663 6852.264   663
loyal_customers      36.287   742     6.830   742 2746.067   742
need_attention       53.266   207     2.449   207 1060.357   207
new_customers         8.580    50     1.000    50  386.199    50
potential_loyalists  18.793   517     2.017   517  729.511   517
promising            25.747    87     1.000    87  367.087    87

In [52]:
# belirli bir sınıfa ( need attention) erişmek istersek
rfm[rfm["segment"]== "need_attention"].head()

,Recency,Frequency,Monetary,Recency_score,Frequency_score,Monetary_score,RFM_SCORE,segment
Customer ID,,,,,,,,
12349.000,43,3,2671.140,3,3,5,33,need_attention
12369.000,49,3,1791.150,3,3,4,33,need_attention
12371.000,45,3,2179.420,3,3,5,33,need_attention
12374.000,57,3,2246.290,3,3,5,33,need_attention
12389.000,38,3,1433.330,3,3,4,33,need_attention


In [55]:
# belirli bir sınıfa ( cant_loose) erişmek istersek
rfm[rfm["segment"] == "Cant_loose"].head()

,Recency,Frequency,Monetary,Recency_score,Frequency_score,Monetary_score,RFM_SCORE,segment
Customer ID,,,,,,,,
12346.000,165,11,372.860,2,5,2,25,Cant_loose
12380.000,101,7,6951.490,2,5,5,25,Cant_loose
12482.000,212,29,23691.400,1,5,5,15,Cant_loose
12510.000,95,7,4195.450,2,5,5,25,Cant_loose
12891.000,94,8,509.500,2,5,3,25,Cant_loose


In [57]:
# kaybedemeyeceğimiz müşterilerinindex bilgisi
rfm[rfm["segment"] == "Cant_loose"].index

Float64Index([12346.0, 12380.0, 12482.0, 12510.0, 12891.0, 12932.0, 13044.0,
              13313.0, 13680.0, 13782.0, 13799.0, 13856.0, 14025.0, 14063.0,
              14160.0, 14221.0, 14548.0, 14607.0, 14685.0, 14745.0, 15003.0,
              15013.0, 15015.0, 15125.0, 15141.0, 15222.0, 15306.0, 15321.0,
              15359.0, 15369.0, 15372.0, 15443.0, 15538.0, 15607.0, 15633.0,
              15722.0, 15751.0, 15754.0, 15768.0, 15911.0, 15912.0, 16027.0,
              16032.0, 16158.0, 16177.0, 16197.0, 16335.0, 16467.0, 16631.0,
              16742.0, 16743.0, 16875.0, 16986.0, 17021.0, 17032.0, 17092.0,
              17113.0, 17157.0, 17188.0, 17230.0, 17268.0, 17426.0, 17448.0,
              17454.0, 17512.0, 17578.0, 17602.0, 17651.0, 17940.0, 17969.0,
              17988.0, 18009.0, 18051.0, 18064.0, 18094.0, 18251.0, 18258.0],
             dtype='float64', name='Customer ID')

In [64]:
#  can_loose indexlerini kullanılabilir olarak dışarıçıkaralım
new_df =pd.DataFrame()
new_df["new_customer_id"] =rfm[rfm["segment"]== "new_customers"].index

In [66]:
new_df["new_customer_id"] = rfm[rfm["segment"] == "new_customers"].index
new_df.head()

,new_customer_id
0,12351.000
1,12385.000
2,12386.000
3,12427.000
4,12441.000


In [67]:
# new id değerlerinin sonundaki  3 adet sıfır işimize yaramıyor. Kurtulalım...
new_df = new_df["new_customer_id"].astype(int)
new_df.head()

0    12351
1    12385
2    12386
3    12427
4    12441
Name: new_customer_id, dtype: int64

In [71]:
# oluşturduğumuz new_customer_id değerlerinin bir csv dosyasına çevirelim
new_df.to_csv("new_customers.csv")

In [72]:
# bütün segment dosyasını da oluşturabiliriz
rfm.to_csv("rfm.csv")